# Quick start guide for Anafibre
This notebook provides a quick start guide to using Anafibre. It covers the basics of:
- creating a fibre, 
- calculating its modes, and 
- visualising the results.

We will start by importing the necessary libraries and setting up our environment:

In [1]:
import numpy as np          # For numerical operations
import anafibre as fib      # For fibre optics calculations

## Create a step-index fibre
Next, we will create a step-index fibre and choose a wavelength of 700 nm for the mode calculations.

In [ ]:
fibre = fib.StepIndexFibre(
    core_radius=250e-9, # Core radius of 250 nm
    n_core=2.00,        # Refractive index of the core
    n_clad=1.33)         # Refractive index of the cladding (e.g., water)

wl = 700e-9     # Wavelength of 700 nm

Notice that all the parameters are specified in SI units such as the core radius and wavelength in meters. Refractive indices `n_core` and `n_clad` can be specified as functions of wavelength, or as constants like in the example above. If we wanted a function we could pass `n_core=lambda x: 2.00+0.01*(x-700e-9)` for example, which would give a core refractive index that increases linearly with wavelength. One can also instead specify the relative `eps_core` and `eps_clad` parameters, and `mu_core` and `mu_clad` if the fibre is magnetic. The fibre object will automatically calculate the missing parameters from the ones that are provided. One can call fibre parameters at any wavelength as shown below:

In [3]:
print(f"Fibre parameters at λ = {wl*1e9:.0f} nm:")
print(f"  Core radius: {fibre.core_radius*1e9:.0f} nm")
print(f"  n_core:      {fibre.n_core(wl):.2f}")
print(f"  n_clad:      {fibre.n_clad(wl):.2f}")
print(f"\nMaterial properties:")
print(f"  ε_core:      {fibre.eps_core(wl):.2f}")
print(f"  ε_clad:      {fibre.eps_clad(wl):.2f}")
print(f"  μ_core:      {fibre.mu_core(wl):.2f}")
print(f"  μ_clad:      {fibre.mu_clad(wl):.2f}")
print(f"\nV-number at λ = {wl*1e9:.0f} nm: {fibre.V(wl):.3f}")

Fibre parameters at λ = 700 nm:
  Core radius: 250 nm
  n_core:      2.00
  n_clad:      1.33

Material properties:
  ε_core:      4.00
  ε_clad:      1.77
  μ_core:      1.00
  μ_clad:      1.00

V-number at λ = 700 nm: 3.352


## Calculate guided modes
The fibre class also  contains **mode constructor** methods for HE<sub>ℓn&nbsp;</sub>, EH<sub>ℓn&nbsp;</sub>, TE<sub>0n&nbsp;</sub>, and TM<sub>0n</sub> modes
>```python
>fibre.HE(ell, n, wl, a_plus=..., a_minus=...)
>fibre.EH(...)
>fibre.TE(n, wl, a=...)
>fibre.TM(...)
>```
Each returns a `GuidedMode` object. So for instance, to calculate the fundamental HE<sub>11</sub> mode at our chosen wavelength and with linear polarisation along the x-axis, we can do:

In [4]:
HE11x = fibre.HE(ell=1, n=1, wl=wl, a_plus=1/np.sqrt(2), a_minus=1/np.sqrt(2))
HE11x

<GuidedMode HE (ℓ=1, n=1), λ=7.00e-07, V=3.35, neff=1.7955, (S₁,S₂,S₃) = 1.00 · (1.00, 0.00, 0.00)>

The mode class provides methods to calculate the fields at any point in space, for example the electric field at the fibre centre:

In [18]:
E_center = HE11x.E(x=0, y=0)
print("Electric field at the fibre center (0,0,0):")
print(f" Ex = {E_center[0].real:+.2e} {E_center[0].imag:+.2e} i")
print(f" Ey = {E_center[1].real:+.2e} {E_center[1].imag:+.2e} i")
print(f" Ez = {E_center[2].real:+.2e} {E_center[2].imag:+.2e} i")
print(f"\n |E| = {np.abs(E_center).sum():.2e} V/m")

Electric field at the fibre center (0,0,0):
 Ex = +0.00e+00 +6.68e+07 i
 Ey = +0.00e+00 +0.00e+00 i
 Ez = +0.00e+00 +0.00e+00 i

 |E| = 6.68e+07 V/m


We can also list all the guided modes at this wavelength using the mode listing method:

In [6]:
modes=fibre.list_modes_at(wl)   # Calculate and display guided modes
modes

[<GuidedMode HE (ℓ=1, n=1), λ=7.00e-07, V=3.35, neff=1.7955, (S₁,S₂,S₃) = 1.00 · (0.00, 0.00, 1.00)>,
 <GuidedMode TM (ℓ=0, n=1), λ=7.00e-07, V=3.35, neff=1.5500, (S₁,S₂,S₃) = 1.00 · (0.00, 0.00, 1.00)>,
 <GuidedMode TE (ℓ=0, n=1), λ=7.00e-07, V=3.35, neff=1.4762, (S₁,S₂,S₃) = 1.00 · (0.00, 0.00, 1.00)>,
 <GuidedMode HE (ℓ=2, n=1), λ=7.00e-07, V=3.35, neff=1.4571, (S₁,S₂,S₃) = 1.00 · (0.00, 0.00, 1.00)>]

Where hybrid modes will default to right-handed circular polarisation. It returns a list `modes` of `GuidedMode` objects. 

Since E and H methods can be calculated at any point in space we can also evaluate them on a grid:

In [26]:
## Calculate field distributions
mode = modes[0]  # Use the first mode from the list
Ngrid = 100  # Number of points in x and y directions
x = np.linspace(-2*fibre.core_radius, 2*fibre.core_radius, Ngrid)
y = np.linspace(-2*fibre.core_radius, 2*fibre.core_radius, Ngrid)
X, Y = np.meshgrid(x, y)   # Create a grid of x and y coordinates

E = mode.E(x=X, y=Y)        # Electric field
H = mode.H(x=X, y=Y)        # Magnetic field
J_E = mode.gradE(x=X, y=Y)  # Gradient of the electric field
J_H = mode.gradH(x=X, y=Y)  # Gradient of the magnetic field

# Print the shapes of the calculated fields and gradients
print(f'Field shapes: \nE:{E.shape}, H:{H.shape}, gradE:{J_E.shape}, gradH:{J_H.shape}')

Field shapes: 
E:(100, 100, 3), H:(100, 100, 3), gradE:(100, 100, 3, 3), gradH:(100, 100, 3, 3)


## Visualise the modes
Finally, we can visualise the modes using the built-in plotting methods. For example, to plot the electric field of the HE<sub>11</sub> mode in the x-y plane at z=0, we can do:

In [ ]:
anim = fib.animate_fields_xy(modes=modes[0], show=("E",),figsize=(5,5))
fib.display_anim(anim)

This animation can also be saved as a video file using the `save` method of the animation object

For example using `ffmpeg` to save the animation as an MP4 file with 30 frames per second, you can do:

```python
anim.save("mode_animation.mp4", writer="ffmpeg", fps=30)
```
Or for a GIF file using the `pillow` writer with 15 frames per second:

```python   
anim.save("mode_animation.gif", writer="pillow", fps=15)
```